In [ ]:
import copy
import itertools
import functools
import collections
import operator
import sys
import re
import numpy as np
import math

In [ ]:
from helpers.functions import *

Configuration

In [ ]:
DIR = "data/2019/"
load_day = functools.partial(load, DIR)

In [ ]:
def intcode_computer(software, idx, *inputs):

    # software = list(map(int, software.split(',')))
    inputs = list(inputs)

    while software[idx] != 99:
        instruction = '{:0>5}'.format(software[idx])
        # print(f"Instruction: {instruction}")
        
        modes = instruction[2::-1]
        assert(all([mod in ["0", "1"] for mod in modes]))
        
        opcode = software[idx] % 100
        
        # position or immediate mode
        if opcode in [1, 2, 4, 5, 6, 7, 8]:
            left_term = software[idx+1] if modes[0] == "1" else software[software[idx+1]]
        if opcode in [1, 2, 5, 6, 7, 8]:
            right_term = software[idx+2] if modes[1] == "1" else software[software[idx+2]]

        if opcode == 1:
            software[software[idx+3]] = left_term + right_term
            idx += 4
        elif opcode == 2:
            software[software[idx+3]] = left_term * right_term
            idx += 4
        elif opcode == 3:
            software[software[idx+1]] = int(inputs.pop(0))
            idx += 2
        elif opcode == 4:
            # print(left_term)
            return {"output": left_term, "idx": idx + 2}
        elif opcode == 5:
            if left_term:
                idx = right_term
            else:
                idx += 3
        elif opcode == 6:
            if not left_term:
                idx = right_term
            else:
                idx += 3
        elif opcode == 7:
            software[software[idx+3]] = int(left_term < right_term)
            idx += 4
        elif opcode == 8:
            software[software[idx+3]] = int(left_term == right_term)
            idx += 4
        else:
            raise ValueError("error, code:", software[idx], " position: ", idx)
        # print(software)

# Problems

## Day 1

http://adventofcode.com/2019/day/1

In [ ]:
content = load_day(1)

In [ ]:
def fuel(mass):
    return mass // 3 - 2

__Part 1__

In [ ]:
total = 0

for elem in map(int, content):
    total += fuel(elem)
    
print(f'Answer 1: {total}')

__Part 2__

In [ ]:
total = 0

for elem in map(int, content):
    fuel_mass = fuel(elem)
    while fuel_mass > 0:
        total += fuel_mass
        fuel_mass = fuel(fuel_mass)

print(f'Answer 2: {total}')

## Day 2

http://adventofcode.com/2019/day/2

In [ ]:
content = load_day(2)
content = list(map(int, content[0].split(',')))

__Part 1__

In [ ]:
def write_software(noun=12, verb=2):
    software = copy.deepcopy(content)
    software[1:3] = [noun, verb]
    return software

In [ ]:
software = write_software()
intcode_computer(software, 0)
print(f'Anwser 1: {software[0]}')

__Part 2__

In [ ]:
for noun in range(100):
    for verb in range(100):
        software = write_software(noun, verb)
        intcode_computer(software, 0)
        if software[0] == 19690720:
            print('Answer 2: {}'.format(noun * 100 + verb))

## Day 3

http://adventofcode.com/2019/day/3

In [ ]:
content = load_day(3)
ex1 = ["R75,D30,R83,U83,L12,D49,R71,U7,L72", "U62,R66,U55,R34,D71,R55,D58,R83"]
ex2 = ["R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51", "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"]

In [ ]:
content = [ c.split(',') for c in content]
ex1 = [ c.split(',') for c in ex1]
ex2 = [ c.split(',') for c in ex2]

In [ ]:
def order_to_coords(value, from_coord):
    orient = value[0]
    length = int(value[1:])
    
    if orient == 'R':
        return [(from_coord[0] + x, from_coord[1]) for x in range(1, length + 1)]
    elif orient == 'L':
        return [(from_coord[0] - x, from_coord[1]) for x in range(1, length + 1)]
    elif orient == 'U':
        return [(from_coord[0], from_coord[1] + x) for x in range(1, length + 1)]
    elif orient == 'D':
        return [(from_coord[0], from_coord[1] - x) for x in range(1, length + 1)]

__Part 1__

In [ ]:
paths = []

for row in content:
    cur_pos = (0, 0)
    path = []

    for order in row:
        points = order_to_coords(order, cur_pos)
        path += points
        
        cur_pos = points[-1]
        
    paths += [path]

In [ ]:
intersections = set(paths[0]).intersection(set(paths[1]))
sorted_intersections = sorted(intersections, key=lambda x: abs(x[0]) + abs(x[1]))
print('Answer 1: {}'.format(abs(sorted_intersections[0][0]) + abs(sorted_intersections[0][1])))

__Part 2__

In [ ]:
distances = []
for inter in intersections:
    distances += [2 + paths[0].index(inter) + paths[1].index(inter)] # initial step for each path is not considered
    
print('Answer 2: {}'.format(min(distances)))

## Day 4

http://adventofcode.com/2019/day/4

In [ ]:
# content = load_day(4)
puzzle_input = "145852-616942"

__Part 1__

In [ ]:
def check_input(value):
    if not isinstance(value, str):
        raise ValueError("input should be string")
        
    elems = list(value)
    
    prev_elem = None
    before_elem = None
    
    adjacent = False
    decrease = False
    
    for e in elems:
        if prev_elem is not None:
            if not adjacent:
                adjacent = prev_elem == e
            if not decrease:
                decrease = ord(prev_elem) > ord(e)

        prev_elem = e
    return adjacent and not decrease

In [ ]:
candidates = []
for v in range(*map(int, puzzle_input.split('-'))):
    inp = str(v)
    candidates += [inp] if check_input(inp) else []
print(f'Answer 1: {len(candidates)}')

__Part 2__

In [ ]:
def lengths(value):
    le = []
    char = ''
    count = 1
    for v in value[1:]:
        if v == char:
            count += 1
        else:
            le.append(count)
            count = 1
            
        char = v
    return 

In [ ]:
for c in candidates:
    
    total += int(check_input2(str(v)))
print(f'Answer 2: {total}')

## Day 5

http://adventofcode.com/2019/day/5

In [ ]:
content = load_day(5)
content = list(map(int, content[0].split(',')))

__Part 1__

In [ ]:
def pass_all_tests(input_value):
    software = copy.deepcopy(content)
    idx = 0

    while software[idx] != 99:
        outputs = intcode_computer(software, idx, input_value)
        idx = outputs["idx"]

    return outputs["output"]

In [ ]:
print(f"Answer 1: {pass_all_tests(1)}")

__Part 2__

In [ ]:
print(f"Answer 2: {pass_all_tests(5)}")

## Day 6

http://adventofcode.com/2019/day/6

In [ ]:
content = load_day(6)

In [ ]:
children = collections.defaultdict(list)
parents = {}

for from_obj, to_obj in map(lambda l: l.split(')'), content):
    children[from_obj] += [to_obj]
    parents[to_obj] = from_obj

__Part 1__

In [ ]:
queue = ['COM']
scores = {'COM': 0}

while queue:
    obj = queue.pop(0)
    for child in children[obj]:
        scores[child] = scores[obj] + 1
        queue += [child]

print(f"Answer 1: {sum(scores.values())}")

__Part 2__

In [ ]:
def get_path(obj):
    path = []
    while obj != 'COM':
        path += [obj]
        obj = parents[obj]
    return path + ["COM"]

In [ ]:
you_path = get_path("YOU")
san_path = get_path("SAN")

common_stars = set(you_path) & set(san_path)

for idx in range(len(you_path)):
    if you_path[idx] in common_stars:
        you_length = idx
        break

for idx in range(len(san_path)):
    if san_path[idx] in common_stars:
        san_length = idx
        break

print(f"Answer 2: {you_length + san_length - 2}")  # first and last "transfers" do not count

## Day 7

http://adventofcode.com/2019/day/7

In [ ]:
content = load_day(7)
content = list(map(int, content[0].split(',')))

__Part 1__

In [ ]:
max_output = 0

for perm in map(list, itertools.permutations(range(5))):
    input_val = 0

    while perm:
        software = copy.deepcopy(content)
        inputs = [perm.pop(0), input_val]
        outputs = intcode_computer(software, 0, *inputs)
        input_val = outputs["output"]

    if input_val > max_output:
        max_output = input_val
    
print(f"Anwser 1: {max_output}")

__Part 2__

In [ ]:
max_output = 0

for perm in map(list, itertools.permutations(range(5, 10))):
    softwares = [content] * 5
    pointers = [0] * 5
    outputs = {}

    ampli_idx = 0
    input_val = 0
    
    stored_value = 0
    
    while True:
        inputs = [softwares[ampli_idx], pointers[ampli_idx], input_val]

        if perm:
            inputs.insert(2, perm.pop(0))

        outputs = intcode_computer(*inputs)

        if softwares[ampli_idx][pointers[ampli_idx]] != 99:
            input_val = outputs["output"]
            
            pointers[ampli_idx] = outputs["idx"]

            if ampli_idx == 4:
                stored_value = input_val

            ampli_idx = (ampli_idx + 1) % 5
        else:
            break

    if stored_value > max_output:
        max_output = stored_value
print(f"Anwser 2: {max_output}")

## Day 8

http://adventofcode.com/2019/day/8

In [ ]:
content = load_day(8)[0]
width, height = 25, 6

__Part 1__

In [ ]:
img_size = width * height
N = len(content)

images = []

min_zero = img_size
result = 0

for idx in range(0, N, img_size):
    layer = list(content[idx:idx+img_size])
    counter = collections.Counter(layer)
    
    if counter['0'] < min_zero:
        min_zero = counter['0']
        result = counter['1'] * counter['2']
    
    images += [ np.array(layer) ]
        
print(f"Answer 1: {result}")

__Part 2__

In [ ]:
result_image = np.empty(img_size, dtype="str")

for img in images[::-1]:
    result_image[img == "1"] = "x"
    result_image[img == "0"] = " "
    
res = result_image.tolist()
print("\n".join([''.join(res[width*i:width*(i+1)]) for i in range(height)]))

## Day 9

http://adventofcode.com/2019/day/9

In [ ]:
content = load_day(9)

__Part 1__

__Part 2__

## Day 10

http://adventofcode.com/2019/day/10

In [ ]:
content = load_day(10)

__Part 1__

__Part 2__

## Day 11

http://adventofcode.com/2019/day/11

In [ ]:
content = load_day(11)

__Part 1__

__Part 2__

## Day 12

http://adventofcode.com/2019/day/12

In [ ]:
content = load_day(12)

__Part 1__

__Part 2__

## Day 13

http://adventofcode.com/2019/day/13

In [ ]:
content = load_day(13)

__Part 1__

__Part 2__

## Day 14

http://adventofcode.com/2019/day/14

In [ ]:
content = load_day(14)

__Part 1__

__Part 2__

## Day 15

http://adventofcode.com/2019/day/15

In [ ]:
content = load_day(15)

__Part 1__

__Part 2__

## Day 16

http://adventofcode.com/2019/day/16

In [ ]:
content = load_day(16)

__Part 1__

__Part 2__

## Day 17

http://adventofcode.com/2019/day/17

In [ ]:
content = load_day(17)

__Part 1__

__Part 2__

## Day 18

http://adventofcode.com/2019/day/18

In [ ]:
content = load_day(18)

__Part 1__

__Part 2__

## Day 19

http://adventofcode.com/2019/day/19

In [ ]:
content = load_day(19)

__Part 1__

__Part 2__

## Day 20

http://adventofcode.com/2019/day/20

In [ ]:
content = load_day(20)

__Part 1__

__Part 2__

## Day 21

http://adventofcode.com/2019/day/21

In [ ]:
content = load_day(21)

__Part 1__

__Part 2__

## Day 22

http://adventofcode.com/2019/day/22

In [ ]:
content = load_day(22)

__Part 1__

__Part 2__

## Day 23

http://adventofcode.com/2019/day/23

In [ ]:
content = load_day(23)

__Part 1__

__Part 2__

## Day 24

http://adventofcode.com/2019/day/24

In [ ]:
content = load_day(24)

__Part 1__

__Part 2__

## Day 25

http://adventofcode.com/2019/day/25

In [ ]:
content = load_day(25)

__Part 1__

__Part 2__